In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import optuna
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from imblearn.over_sampling import SMOTE

In [3]:
# Define the function to read OMG data from a CSV file
def read_omg_csv(path_palm_data: str, 
                 n_omg_channels: int, 
                 n_acc_channels: int = 0, 
                 n_gyr_channels: int = 0, 
                 n_mag_channels: int = 0, 
                 n_enc_channels: int = 0,
                 button_ch: bool = True, 
                 sync_ch: bool = True, 
                 timestamp_ch: bool = True) -> pd.DataFrame:
    
    df_raw = pd.read_csv(path_palm_data, sep=' ', 
                         header=None, 
                         skipfooter=1, 
                         skiprows=1, 
                         engine='python')
    columns = np.arange(n_omg_channels).astype('str').tolist()
    
    for label, label_count in zip(['ACC', 'GYR', 'MAG', 'ENC'], 
                                  [n_acc_channels, n_gyr_channels, n_mag_channels, n_enc_channels]):
        columns = columns + ['{}{}'.format(label, i) for i in range(label_count)]
        
    if button_ch:
        columns = columns + ['BUTTON']
        
    if sync_ch:
        columns = columns + ['SYNC']
        
    if timestamp_ch:
        columns = columns + ['ts']
        
    df_raw.columns = columns
    
    return df_raw

In [4]:
def prepare_training_data(path_palm_data, path_protocol_data, path_meta_data, 
                          n_omg_channels=50, n_acc_channels=3, n_gyr_channels=3, 
                          n_mag_channels=0, n_enc_channels=6, 
                          standardize=True, normalize=True,
                          DO_REPLACE_TO_MOVING_AVERAGE=True, 
                          DO_CALCULATE_DERIVATIVE=True,
                          DO_SHIFT_GESTURE=True,
                          selected_channels='ALL'):
    """
    Подготовка данных для обучения и тестирования из файлов данных palm, protocol и meta.
    
    Аргументы:
    path_palm_data (str): Путь к файлу данных palm.
    path_protocol_data (str): Путь к файлу данных protocol.
    path_meta_data (str): Путь к файлу данных meta.
    n_omg_channels, n_acc_channels, и т.д. (int): Количество каналов сенсоров.
    standardize (bool): Если True, стандартизирует признаки.
    normalize (bool): Если True, нормализует признаки.
    DO_REPLACE_TO_MOVING_AVERAGE (bool): Если True, применяет скользящее среднее к данным OMG.
    DO_CALCULATE_DERIVATIVE (bool): Если True, вычисляет производные данных OMG.
    DO_SHIFT_GESTURE (bool): Если True, смещает целевой признак на максимальный скачок в данных.
    selected_channels (str): Выбор каналов данных ('OMG', 'ACC_GYR', 'ALL').
    
    Возвращает:
    tuple: Кортеж, содержащий данные для обучения и тестирования.
    """
    # Чтение данных OMG
    omg_data = read_omg_csv(path_palm_data, n_omg_channels, n_acc_channels, n_gyr_channels, 
                            n_mag_channels, n_enc_channels)
    
    # Чтение данных протокола и кодирование жестов
    gestures_protocol = pd.read_csv(path_protocol_data)
    le = LabelEncoder()
    gestures_protocol['gesture'] = le.fit_transform(
        gestures_protocol[[
            "Thumb", "Index", "Middle", "Ring", "Pinky",
            'Thumb_stretch', 'Index_stretch', 'Middle_stretch', 'Ring_stretch', 'Pinky_stretch'
        ]].apply(lambda row: str(tuple(row)), axis=1)
    )
    
    # Чтение метаинформации
    df_meta = pd.read_csv(path_meta_data)
    palm_file = path_palm_data.split('/')[-1]
    last_train_idx = df_meta[df_meta['montage'] == palm_file].to_dict(orient='records')[0]['last_train_idx']
    
    # Синхронизация меток жестов с данными OMG, используя канал SYNC
    y_cmd = np.array([gestures_protocol['gesture'].loc[s] for s in omg_data['SYNC'].values])
    
    # Подготовка названий признаков для данных OMG
    OMG_CH = [str(i) for i in range(n_omg_channels)]
    ACC_CH = ['ACC0', 'ACC1', 'ACC2']
    GYR_CH = ['GYR0', 'GYR1', 'GYR2']
    ALL_CH = OMG_CH + ACC_CH + GYR_CH

    # Выбор каналов в соответствии с параметром selected_channels
    if selected_channels == 'OMG':
        selected_features = OMG_CH
    elif selected_channels == 'ACC_GYR':
        selected_features = ACC_CH + GYR_CH
    else:
        selected_features = ALL_CH
    
    if DO_REPLACE_TO_MOVING_AVERAGE:
        # Замена на скользящее среднее
        for col in selected_features:
            omg_data[col] = omg_data[col].rolling(window=5).mean().bfill()
    
    if DO_CALCULATE_DERIVATIVE:
        # Вычисление производных данных
        OMG_DERIV = [f'{col}_deriv' for col in OMG_CH]
        for col in OMG_CH:
            omg_data[f'{col}_next'] = omg_data[col].shift(-1).ffill()
            omg_data[f'{col}_deriv'] = omg_data[f'{col}_next'] - omg_data[col]
        selected_features += OMG_DERIV

    if DO_SHIFT_GESTURE:
        # Смещение целевого признака
        id_max = 0
        cur_gesture = 0
        for i in range(y_cmd.shape[0]):
            if i < id_max:  # Пропускаем все значения до id_max
                continue
            prev_gesture = cur_gesture  # предыдущий жест
            cur_gesture = y_cmd[i]  # текущий жест
            if cur_gesture != prev_gesture:  # Если сменился жест
                id_max = omg_data[OMG_DERIV][i:i+35].abs().sum(axis=1).idxmax()  # Нахождение максимального скачка
                y_cmd[i:id_max] = prev_gesture  # Замена всех значений до id_max на предыдущий жест
    
    # Разделение данных на обучающие и тестовые наборы
    X_train = omg_data[selected_features].iloc[:last_train_idx].values
    y_train = y_cmd[:last_train_idx]
    X_test = omg_data[selected_features].iloc[last_train_idx:].values
    y_test = y_cmd[last_train_idx:]
    
    # Стандартизация и нормализация
    if standardize:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
    if normalize:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    return (X_train, y_train), (X_test, y_test)

path_palm_data = 'data/2023-05-31_17-14-41.palm'
path_protocol_data = 'data/2023-05-31_17-14-41.palm.protocol.csv'
path_meta_data = 'data/meta_information.csv'

(X_train, y_train), (X_test, y_test) = prepare_training_data(path_palm_data, path_protocol_data, path_meta_data, standardize=True, normalize=False)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15679, 106), (15679,), (3889, 106), (3889,))

In [5]:
# Посчитаем, как распределены классы
class_counts_train = Counter(y_train)
class_counts_test = Counter(y_test)

class_counts_train, class_counts_test

(Counter({0: 10677, 5: 1052, 3: 1032, 4: 1020, 1: 952, 2: 946}),
 Counter({0: 2608, 2: 279, 5: 258, 3: 252, 1: 246, 4: 246}))

Как видно, классы не сбалансированы: класс 0 значительно превосходит по количеству остальные классы в обоих наборах данных. 

In [10]:
# Инициализация экземпляра SMOTE
smote = SMOTE(k_neighbors=4, sampling_strategy='auto', random_state=42)

# Применение SMOTE к тренировочным данным
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Подсчёт количества примеров каждого класса после применения SMOTE
class_counts_train_smote = Counter(y_train_smote)

print(class_counts_train_smote)

Counter({0: 10677, 3: 10677, 5: 10677, 1: 10677, 4: 10677, 2: 10677})


In [11]:
def build_and_train_model(X_train, y_train, X_test, y_test, epochs=100, batch_size=50):
    num_classes = len(np.unique(y_train))  # Determine the number of unique classes

    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    

    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

    return model, history, test_loss, test_accuracy

# Example usage:
model, history, test_loss, test_accuracy = build_and_train_model(X_train_smote, y_train_smote, X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

Epoch 1/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6472 - loss: 1.0109 - val_accuracy: 0.9855 - val_loss: 0.0450
Epoch 2/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9387 - loss: 0.1796 - val_accuracy: 0.9920 - val_loss: 0.0216
Epoch 3/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9695 - loss: 0.0981 - val_accuracy: 0.9881 - val_loss: 0.0293
Epoch 4/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9788 - loss: 0.0719 - val_accuracy: 0.9958 - val_loss: 0.0100
Epoch 5/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9807 - loss: 0.0647 - val_accuracy: 0.9952 - val_loss: 0.0112
Epoch 6/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9840 - loss: 0.0539 - val_accuracy: 0.9991 - val_loss: 0.0038
Epoch 7/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9856 - loss: 0.0468 - val_accuracy: 0.9874 - val_loss: 0.0318
Epoch 8/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9861 - loss: 0

In [8]:
def objective(trial):
    # Предлагаем параметры
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.7)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = 100  # Можно также оптимизировать количество эпох

    # Создание модели
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
    
    # Компиляция модели
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Обучение модели
    model.fit(X_train_smote, y_train_smote, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    
    # Оценка модели
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy  # Максимизация точности

# Создание исследования
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=-1)  # Можно изменить количество испытаний в зависимости от времени/ресурсов

print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-05-07 20:32:28,515] A new study created in memory with name: no-name-cb9b09b8-03d5-4375-bf0d-4e5f2ef7d4cc
[I 2024-05-07 20:38:45,789] Trial 14 finished with value: 0.9382874965667725 and parameters: {'learning_rate': 2.9958343372419382e-05, 'dropout_rate': 0.6636902867318807, 'batch_size': 256}. Best is trial 14 with value: 0.9382874965667725.
[I 2024-05-07 20:38:48,194] Trial 22 finished with value: 0.9485728740692139 and parameters: {'learning_rate': 0.0019660620328689304, 'dropout_rate': 0.29631700860217425, 'batch_size': 256}. Best is trial 22 with value: 0.9485728740692139.
[I 2024-05-07 20:38:48,735] Trial 21 finished with value: 0.95140141248703 and parameters: {'learning_rate': 0.00017833008390955146, 'dropout_rate': 0.572273503996012, 'batch_size': 256}. Best is trial 21 with value: 0.95140141248703.
[I 2024-05-07 20:38:49,022] Trial 11 finished with value: 0.9429159164428711 and parameters: {'learning_rate': 0.0551246872605571, 'dropout_rate': 0.007072490964556832, 'b

Best trial:
  Value: 0.9547441601753235
  Params: 
    learning_rate: 3.754621789416905e-05
    dropout_rate: 0.40807076459005276
    batch_size: 64


In [9]:
# Загрузка лучшей модели
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(trial.params['dropout_rate']),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(trial.params['dropout_rate']),
    Dense(len(np.unique(y_train)), activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=trial.params['learning_rate']),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Повторное обучение лучшей модели
model.fit(X_train_smote, y_train_smote, epochs=100, batch_size=trial.params['batch_size'], verbose=0)

# Предсказания
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Отчёт о классификации
report = classification_report(y_test, y_pred_classes)
print(report)

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      2608
           1       0.98      0.98      0.98       246
           2       0.93      0.99      0.95       279
           3       0.77      0.79      0.78       252
           4       0.87      0.97      0.92       246
           5       0.94      0.95      0.94       258

    accuracy                           0.95      3889
   macro avg       0.91      0.94      0.92      3889
weighted avg       0.95      0.95      0.95      3889



In [12]:
def build_and_train_model(X_train, y_train, X_test, y_test, trial):
    # Параметры для SMOTE
    k_neighbors = trial.suggest_int('k_neighbors', 2, 10)
    sampling_strategy = trial.suggest_categorical('sampling_strategy', ['auto', 'minority', 'not majority', 'all'])
    
    # Применяем SMOTE
    smote = SMOTE(k_neighbors=k_neighbors, sampling_strategy=sampling_strategy)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    # Параметры модели
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.7)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = 100

    num_classes = len(np.unique(y_resampled))
    
    # Создание модели
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_resampled.shape[1],)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    
    # Компиляция модели
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Обучение модели
    history = model.fit(X_resampled, y_resampled, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    
    # Оценка модели
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    return test_accuracy

def objective(trial):
    return build_and_train_model(X_train, y_train, X_test, y_test, trial)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Лучшие параметры:")
print(study.best_trial)

[I 2024-05-07 21:18:02,341] A new study created in memory with name: no-name-d68c4678-9c26-460b-87fe-8ed9a5a793d5
[I 2024-05-07 21:18:47,622] Trial 0 finished with value: 0.9393160343170166 and parameters: {'k_neighbors': 7, 'sampling_strategy': 'minority', 'learning_rate': 0.00228183750225705, 'dropout_rate': 0.2757413619530013, 'batch_size': 64}. Best is trial 0 with value: 0.9393160343170166.
[I 2024-05-07 21:20:35,064] Trial 1 finished with value: 0.9398303031921387 and parameters: {'k_neighbors': 10, 'sampling_strategy': 'all', 'learning_rate': 1.164216277565633e-05, 'dropout_rate': 0.6605432548210465, 'batch_size': 64}. Best is trial 1 with value: 0.9398303031921387.
[I 2024-05-07 21:22:22,644] Trial 2 finished with value: 0.9483157396316528 and parameters: {'k_neighbors': 9, 'sampling_strategy': 'not majority', 'learning_rate': 0.016969931864945123, 'dropout_rate': 0.5189124851750998, 'batch_size': 64}. Best is trial 2 with value: 0.9483157396316528.
[I 2024-05-07 21:24:59,638] 